### 2024 A

##### Problem 1

In [ ]:
import numpy as np
import math 
from scipy.optimize import brentq

# =================================== const & input ===================================
t_globaler=395
max_theta=1000*math.pi
p0=0.55

# =================================== functions ===================================
class length:
    def __init__(self,p):
        self.pi=np.pi
        self.k=p/(2*self.pi)
        self.r0=1.65
        self.r1=2.86
        self.totallength=self.length(32*self.pi,0)
    
    def getr(self,theta):
        return self.k*theta

    def length(self,up,down):
        lup=(self.k/2)*(up*math.sqrt(up**2+1)+math.log(up+math.sqrt(up**2+1)))
        ldown=(self.k/2)*(down*math.sqrt(down**2+1)+math.log(down+math.sqrt(down**2+1)))
        return lup-ldown
    
class equation(length):
    def __init__(self,p,t_global):
        self.pi=np.pi
        self.t=t_global
        super().__init__(p)

    def dragonhead(self,theta0):
        ls1=self.totallength-self.t
        rs1=self.length(theta0,0)
        return ls1-rs1

# =================================== output theta_0 ===================================
lengther= length(p0)
equationer=equation(p0,t_globaler)
print(equationer.dragonhead(0))
print(equationer.dragonhead(max_theta))

theta_0 = brentq(equationer.dragonhead, a=0, b=max_theta, xtol=1e-8)
print(f"theta[0] = {theta_0:.6f}")  
x_0=lengther.k*theta_0*math.cos(theta_0)
y_0=lengther.k*theta_0*math.sin(theta_0)
print(f"x[0] = {x_0:.6f}")
print(f"y[0] = {y_0:.6f}")
theta=[]
x=[]
y=[]
theta.append(theta_0)
x.append(x_0)
y.append(y_0)
v_0 = 1 / (lengther.k * math.sqrt(1+theta_0**2))
v=[]
v.append(v_0)

# ================================== output x,y[] ===================================
for i in range(1, 224):

    def recursive_equation(theta_next):
        ls2=(lengther.getr(theta[i-1]))**2+(lengther.getr(theta_next))**2-2*lengther.getr(theta[i-1])*lengther.getr(theta_next)*math.cos(theta[i-1]-theta_next)
        rs2=lengther.r0**2 if i!=1 else lengther.r1**2
        return ls2-rs2

    theta_next = brentq(recursive_equation, theta[i-1], max_theta, xtol=1e-8)
    theta.append(theta_next)
    x.append(lengther.getr(theta_next)*math.cos(theta_next))
    y.append(lengther.getr(theta_next)*math.sin(theta_next))

    vup = theta[i-1] - theta[i] * math.cos(theta[i] - theta[i-1]) - theta[i] * theta[i-1] * math.sin(theta[i] - theta[i-1])
    vdown = theta[i-1] * math.cos(theta[i] - theta[i-1]) - theta[i] - theta[i] * theta[i-1] * math.sin(theta[i] - theta[i-1])
    v.append((vup / vdown) * v[i-1])

for i in range(0,224):
    v[i]=v[i]*math.sqrt(1+theta[i]**2)*lengther.k

print(f"x[1] = {x[1]:.6f}")
print(f"y[1] = {y[1]:.6f}")
print(f"x[51] = {x[51]:.6f}")
print(f"y[51] = {y[51]:.6f}")
print(f"x[101] = {x[101]:.6f}")
print(f"y[101] = {y[101]:.6f}")
print(f"x[151] = {x[151]:.6f}")
print(f"y[151] = {y[151]:.6f}")
print(f"x[201] = {x[201]:.6f}")
print(f"y[201] = {y[201]:.6f}")
print(f"x[223] = {x[223]:.6f}")
print(f"y[223] = {y[223]:.6f}")
print(f"v[0] = {v[0]:.6f}")
print(f"v[1] = {v[1]:.6f}")
print(f"v[51] = {v[51]:.6f}")
print(f"v[101] = {v[101]:.6f}")
print(f"v[151] = {v[151]:.6f}")
print(f"v[201] = {v[201]:.6f}")
print(f"v[223] = {v[223]:.6f}")

# ================================== write csv ===================================
import csv
with open("C:\\Users\\32231\\Desktop\\GJSB\\MATLAB\\P1.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    for i in range(len(x)):
        writer.writerow([f'x[{i}]', f'{x[i]:.6f}'])
        writer.writerow([f'y[{i}]', f'{y[i]:.6f}'])

# print("数据已成功写入dragon_curve_coordinates.csv文件")




#### Problem 2

##### 可视化搜索

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon


csv_path = "C:\\Users\\32231\\Desktop\\GJSB\\MATLAB\\P1.csv"  

x_coords = []
y_coords = []

with open(csv_path, 'r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    for idx, row in enumerate(reader):
        value = float(row[1])
        if idx % 2 == 0:
            x_coords.append(value)
        else:
            y_coords.append(value)

num_points = len(x_coords)

plt.figure(figsize=(6, 5))
ax = plt.gca()
ax.set_aspect('equal') 
ax.axis('off')
ax.plot(x_coords, y_coords, 'ro', markersize=5)


vertical_distance = 0.15  
extend_length = 0.275     


for i in range(num_points - 1):
    
    xA, yA = x_coords[i], y_coords[i]
    xB, yB = x_coords[i+1], y_coords[i+1]
    
    dx = xB - xA
    dy = yB - yA
    segment_length = np.sqrt(dx**2 + dy**2)
    ux = dx / segment_length  
    uy = dy / segment_length  
    xA_extend = xA - ux * extend_length
    yA_extend = yA - uy * extend_length
    xB_extend = xB + ux * extend_length
    yB_extend = yB + uy * extend_length
    
    ax.plot([xA_extend, xB_extend], [yA_extend, yB_extend], 'm--', linewidth=1)
    
    vx1, vy1 = -uy, ux   
    vx2, vy2 = uy, -ux   
    p1 = (xA_extend + vertical_distance * vx1, yA_extend + vertical_distance * vy1)
    p2 = (xB_extend + vertical_distance * vx1, yB_extend + vertical_distance * vy1)
    p3 = (xB_extend + vertical_distance * vx2, yB_extend + vertical_distance * vy2)
    p4 = (xA_extend + vertical_distance * vx2, yA_extend + vertical_distance * vy2)
    
    rectangle = Polygon([p1, p2, p3, p4], 
                       edgecolor='green', 
                       facecolor='green', 
                       linewidth=1, 
                       alpha=0.3)
    ax.add_patch(rectangle)

plt.tight_layout(pad=0)
plt.show()


##### 粗略搜索和中精度排除、包含了最后的搜索过程（瞪眼法）

In [ ]:
t_start=412.47381 #390
t_end=412.47386 #420
step=0.000001 #0.5
min_lengths=[]

for t in np.arange(t_start, t_end, step):
    print(t)
    distances=[]
    able_distances=[]
    # =================================== output theta_0 ===================================
    lengther= length(p0)
    equationer=equation(p0,t)
    # print(equationer.dragonhead(0))
    # print(equationer.dragonhead(max_theta))

    theta_0 = brentq(equationer.dragonhead, a=0, b=max_theta, xtol=1e-8)
    print(f"theta[0] = {theta_0:.6f}")  
    x_0=lengther.k*theta_0*math.cos(theta_0)
    y_0=lengther.k*theta_0*math.sin(theta_0)
    print(f"x[0] = {x_0:.6f}")
    print(f"y[0] = {y_0:.6f}")
    theta=[]
    x=[]
    y=[]
    theta.append(theta_0)
    x.append(x_0)
    y.append(y_0)
    print(theta[0])


    # ================================== output x,y[] ===================================
    for i in range(1, 224):

        
        def recursive_equation(theta_next):
            ls2=(lengther.getr(theta[i-1]))**2+(lengther.getr(theta_next))**2-2*lengther.getr(theta[i-1])*lengther.getr(theta_next)*math.cos(theta[i-1]-theta_next)
            rs2=lengther.r0**2 if i!=1 else lengther.r1**2
            return ls2-rs2

        theta_next = brentq(recursive_equation, theta[i-1], max_theta, xtol=1e-8)

        theta.append(theta_next)
        x.append(lengther.getr(theta_next)*math.cos(theta_next))
        y.append(lengther.getr(theta_next)*math.sin(theta_next))
        if theta_next>= theta[0]+2.5*math.pi:
            break

    # ================================== plt(test1) & length(test2) ===================================
    plt.figure(figsize=(12, 10))
    ax = plt.gca()
    ax.set_aspect('equal') 
    ax.axis('off')

    for i in range(len(x)-1):

        if (theta[0]+(7*math.pi)/4 < theta[i] < theta[0]+2*math.pi or i==0 
            or theta[1]+(7*math.pi)/4 < theta[i] < theta[1]+2*math.pi ):
        
            xA, yA = x[i], y[i]
            xB, yB = x[i+1], y[i+1]
            
            dx = xB - xA
            dy = yB - yA
            segment_length = np.sqrt(dx**2 + dy**2)
            ux = dx / segment_length  
            uy = dy / segment_length  
            xA_extend = xA - ux * extend_length
            yA_extend = yA - uy * extend_length
            xB_extend = xB + ux * extend_length
            yB_extend = yB + uy * extend_length
            
            ax.plot([xA_extend, xB_extend], [yA_extend, yB_extend], 'm--', linewidth=1)
            
            vx1, vy1 = -uy, ux   
            vx2, vy2 = uy, -ux   
            p1 = (xA_extend + vertical_distance * vx1, yA_extend + vertical_distance * vy1)
            p2 = (xB_extend + vertical_distance * vx1, yB_extend + vertical_distance * vy1)
            p3 = (xB_extend + vertical_distance * vx2, yB_extend + vertical_distance * vy2)
            p4 = (xA_extend + vertical_distance * vx2, yA_extend + vertical_distance * vy2)
            # ax.plot(p4[0], p4[1], 'ro', markersize=5)
            
            rectangle = Polygon([p1, p2, p3, p4], 
                            edgecolor='green', 
                            facecolor='green', 
                            linewidth=1, 
                            alpha=0.3)
            ax.add_patch(rectangle)

            
            x1, y1 = p1
            x2, y2 = p2
            if i == 0:
                x0, y0 = p4[0], p4[1]

            else:
                distance = ((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)/(np.sqrt((y2 - y1)** 2 + (x2 - x1)**2))
                distances.append(distance)
    
    if distances:
        for l in distances:
            if abs(l)<1e-5:able_distances.append(l)

        min_length = min(able_distances)
        # min_lengths.append((t, min_length))
        print(f"t={t:.6f} 时的最小距离: {min_length:.10f}")


    plt.tight_layout(pad=0)
    plt.show()

#### Problem 3

##### 精确化计算 $t$ 的方法

In [ ]:

t_start=412.47381 #390
t_end=412.47386 #420
step=0.000001 #0.5
min_lengths=[]

for t in np.arange(t_start, t_end, step):
    print(t)
    distances=[]
    able_distances=[]
    # =================================== output theta_0 ===================================
    lengther= length(p0)
    equationer=equation(p0,t)
    # print(equationer.dragonhead(0))
    # print(equationer.dragonhead(max_theta))

    theta_0 = brentq(equationer.dragonhead, a=0, b=max_theta, xtol=1e-8)
    print(f"theta[0] = {theta_0:.6f}")  
    x_0=lengther.k*theta_0*math.cos(theta_0)
    y_0=lengther.k*theta_0*math.sin(theta_0)
    print(f"x[0] = {x_0:.6f}")
    print(f"y[0] = {y_0:.6f}")
    theta=[]
    x=[]
    y=[]
    theta.append(theta_0)
    x.append(x_0)
    y.append(y_0)
    print(theta[0])


    # ================================== output x,y[] ===================================
    for i in range(1, 224):

        
        def recursive_equation(theta_next):
            ls2=(lengther.getr(theta[i-1]))**2+(lengther.getr(theta_next))**2-2*lengther.getr(theta[i-1])*lengther.getr(theta_next)*math.cos(theta[i-1]-theta_next)
            rs2=lengther.r0**2 if i!=1 else lengther.r1**2
            return ls2-rs2

        theta_next = brentq(recursive_equation, theta[i-1], max_theta, xtol=1e-8)

        theta.append(theta_next)
        x.append(lengther.getr(theta_next)*math.cos(theta_next))
        y.append(lengther.getr(theta_next)*math.sin(theta_next))
        if theta_next>= theta[0]+2.5*math.pi:
            break

    # ================================== plt(test1) & length(test2) ===================================
    plt.figure(figsize=(12, 10))
    ax = plt.gca()
    ax.set_aspect('equal') 
    ax.axis('off')

    for i in range(len(x)-1):

        if (theta[0]+(7*math.pi)/4 < theta[i] < theta[0]+2*math.pi or i==0 
            or theta[1]+(7*math.pi)/4 < theta[i] < theta[1]+2*math.pi ):
        
            xA, yA = x[i], y[i]
            xB, yB = x[i+1], y[i+1]
            
            dx = xB - xA
            dy = yB - yA
            segment_length = np.sqrt(dx**2 + dy**2)
            ux = dx / segment_length  
            uy = dy / segment_length  
            xA_extend = xA - ux * extend_length
            yA_extend = yA - uy * extend_length
            xB_extend = xB + ux * extend_length
            yB_extend = yB + uy * extend_length
            
            ax.plot([xA_extend, xB_extend], [yA_extend, yB_extend], 'm--', linewidth=1)
            
            vx1, vy1 = -uy, ux   
            vx2, vy2 = uy, -ux   
            p1 = (xA_extend + vertical_distance * vx1, yA_extend + vertical_distance * vy1)
            p2 = (xB_extend + vertical_distance * vx1, yB_extend + vertical_distance * vy1)
            p3 = (xB_extend + vertical_distance * vx2, yB_extend + vertical_distance * vy2)
            p4 = (xA_extend + vertical_distance * vx2, yA_extend + vertical_distance * vy2)
            # ax.plot(p4[0], p4[1], 'ro', markersize=5)
            
            rectangle = Polygon([p1, p2, p3, p4], 
                            edgecolor='green', 
                            facecolor='green', 
                            linewidth=1, 
                            alpha=0.3)
            ax.add_patch(rectangle)

            
            x1, y1 = p1
            x2, y2 = p2
            if i == 0:
                x0, y0 = p4[0], p4[1]

            else:
                distance = ((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)/(np.sqrt((y2 - y1)** 2 + (x2 - x1)**2))
                distances.append(distance)
    
    if distances:
        for l in distances:
            if abs(l)<1e-5:able_distances.append(l)

        min_length = min(able_distances)
        # min_lengths.append((t, min_length))
        print(f"t={t:.6f} 时的最小距离: {min_length:.10f}")


    plt.tight_layout(pad=0)
    plt.show()